# Claude with Extended Thinking: A Comprehensive Tutorial

## What You'll Learn

In this notebook, we'll explore Claude's extended thinking capabilities - a powerful feature that gives Claude enhanced reasoning for complex tasks. We'll start with the basics and gradually build up to advanced use cases.

### Table of Contents
1. [Introduction to Extended Thinking](#introduction)
2. [Setting Up Your Environment](#setup)
3. [Basic Usage](#basic-usage)
4. [Understanding Thinking Blocks](#thinking-blocks)
5. [Advanced Features](#advanced-features)
6. [Best Practices](#best-practices)
7. [Real-World Examples](#examples)
8. [Performance and Cost Considerations](#performance)

<a id='introduction'></a>
## 1. Introduction to Extended Thinking

Extended thinking is a feature that allows Claude to "think" through complex problems step-by-step before providing a final answer. This is particularly useful for:

- 🧮 **Mathematical problems** requiring multi-step calculations
- 🔍 **Complex analysis** of documents or data
- 🏗️ **Architecture decisions** in software development
- 🎯 **Strategic planning** and decision-making

### How It Works

When extended thinking is enabled, Claude:
1. Creates internal "thinking" content blocks
2. Works through the problem systematically
3. Incorporates insights from this reasoning
4. Delivers a more thoughtful final response

<a id='setup'></a>
## 2. Setting Up Your Environment

Let's start by installing the necessary packages and setting up our API key.

In [ ]:
# Install the Anthropic Python SDK
!pip install anthropic>=0.34.0

In [ ]:
import os
import anthropic
from IPython.display import Markdown, display
import json
import time

# Set up your API key
# You can either set it as an environment variable or directly here
# For security, we recommend using environment variables
api_key = os.getenv('ANTHROPIC_API_KEY')
if not api_key:
    api_key = input("Please enter your Anthropic API key: ")

client = anthropic.Anthropic(api_key=api_key)

### Supported Models

Extended thinking is supported in:
- **Claude Opus 4** (`claude-opus-4-20250514`)
- **Claude Sonnet 4** (`claude-sonnet-4-20250514`)
- **Claude Sonnet 3.7** (`claude-3-7-sonnet-20250219`)

Note: Claude 4 models return summarized thinking, while Claude 3.7 returns full thinking output.

<a id='basic-usage'></a>
## 3. Basic Usage

Let's start with a simple example to see extended thinking in action.

In [ ]:
def basic_thinking_example():
    """A simple example demonstrating extended thinking"""
    
    response = client.messages.create(
        model="claude-sonnet-4-20250514",
        max_tokens=2000,
        thinking={
            "type": "enabled",
            "budget_tokens": 5000  # How many tokens Claude can use for thinking
        },
        messages=[{
            "role": "user",
            "content": "What is 27 * 453? Show me how you calculate this step by step."
        }]
    )
    
    # Process the response
    for block in response.content:
        if block.type == "thinking":
            print("🤔 Claude's Thinking Process:")
            print("-" * 50)
            print(block.thinking)
            print("-" * 50)
            print()
        elif block.type == "text":
            print("✅ Final Answer:")
            print(block.text)

# Run the example
basic_thinking_example()

### Understanding the Parameters

- **`type: "enabled"`**: Turns on extended thinking
- **`budget_tokens`**: Maximum tokens Claude can use for reasoning
  - Minimum: 1,024 tokens
  - For complex tasks: 16,000+ tokens
  - For very complex tasks: 32,000+ tokens

<a id='thinking-blocks'></a>
## 4. Understanding Thinking Blocks

Let's explore how thinking blocks work and what information they contain.

In [ ]:
def analyze_thinking_blocks():
    """Demonstrate the structure of thinking blocks"""
    
    response = client.messages.create(
        model="claude-sonnet-4-20250514",
        max_tokens=3000,
        thinking={
            "type": "enabled",
            "budget_tokens": 8000
        },
        messages=[{
            "role": "user",
            "content": """I have a list of numbers: [15, 23, 8, 42, 16, 4, 30, 12].
            
            Please:
            1. Find the median
            2. Calculate the mean
            3. Identify any outliers using the IQR method
            4. Suggest what this data might represent"""
        }]
    )
    
    # Analyze the response structure
    print("📊 Response Structure Analysis")
    print("=" * 60)
    
    for i, block in enumerate(response.content):
        print(f"\nBlock {i + 1}:")
        print(f"  Type: {block.type}")
        
        if block.type == "thinking":
            # In Claude 4, thinking is summarized
            print(f"  Thinking Summary Length: {len(block.thinking)} characters")
            print(f"  Has Signature: {'Yes' if hasattr(block, 'signature') else 'No'}")
            print("\n  Thinking Content (first 500 chars):")
            print(f"  {block.thinking[:500]}...")
        elif block.type == "text":
            print(f"  Text Length: {len(block.text)} characters")
            print("\n  Final Response:")
            display(Markdown(block.text))

analyze_thinking_blocks()

### Key Points About Thinking Blocks

1. **Summarization**: Claude 4 models provide summarized thinking, not full reasoning
2. **Billing**: You're charged for full thinking tokens, not the summary
3. **Signature**: Each thinking block includes a cryptographic signature for verification
4. **Privacy**: Thinking blocks help prevent misuse while maintaining intelligence benefits

<a id='advanced-features'></a>
## 5. Advanced Features

### 5.1 Streaming Responses

For better user experience, especially with longer thinking times, you can stream responses.

In [ ]:
def stream_thinking_example():
    """Demonstrate streaming with extended thinking"""
    
    print("🌊 Streaming Extended Thinking Example")
    print("=" * 60)
    
    with client.messages.stream(
        model="claude-opus-4-20250514",
        max_tokens=3000,
        thinking={"type": "enabled", "budget_tokens": 10000},
        messages=[{
            "role": "user",
            "content": """Design a simple REST API for a todo list application. 
            Include endpoints for CRUD operations and consider:
            - Authentication
            - Error handling
            - Data validation
            - Response formats"""
        }],
    ) as stream:
        current_block_type = None
        
        for event in stream:
            if event.type == "content_block_start":
                current_block_type = event.content_block.type
                if current_block_type == "thinking":
                    print("\n🤔 Claude is thinking...", end="", flush=True)
                elif current_block_type == "text":
                    print("\n\n✅ Final Response:\n", end="", flush=True)
            
            elif event.type == "content_block_delta":
                if event.delta.type == "thinking_delta":
                    # Show progress dots for thinking
                    print(".", end="", flush=True)
                elif event.delta.type == "text_delta":
                    print(event.delta.text, end="", flush=True)
            
            elif event.type == "content_block_stop":
                if current_block_type == "thinking":
                    print(" Done thinking!")

stream_thinking_example()

### 5.2 Extended Thinking with Tool Use

Extended thinking can be combined with tool use for even more powerful applications.

In [ ]:
def thinking_with_tools_example():
    """Demonstrate extended thinking with tool use"""
    
    # Define a simple calculator tool
    tools = [{
        "name": "calculator",
        "description": "Perform mathematical calculations",
        "input_schema": {
            "type": "object",
            "properties": {
                "expression": {
                    "type": "string",
                    "description": "Mathematical expression to evaluate"
                }
            },
            "required": ["expression"]
        }
    }]
    
    response = client.messages.create(
        model="claude-sonnet-4-20250514",
        max_tokens=2000,
        thinking={
            "type": "enabled",
            "budget_tokens": 8000
        },
        tools=tools,
        messages=[{
            "role": "user",
            "content": """I'm planning a party for 25 people. Each person will eat:
            - 3 slices of pizza (8 slices per pizza)
            - 2 sodas ($1.50 each)
            - 1 dessert ($3.00 each)
            
            Pizzas cost $12 each. Calculate the total cost and quantities needed."""
        }]
    )
    
    print("🎉 Party Planning with Extended Thinking")
    print("=" * 60)
    
    for block in response.content:
        if block.type == "thinking":
            print("\n🤔 Planning Process:")
            print(block.thinking[:1000] + "...\n")
        elif block.type == "tool_use":
            print(f"\n🔧 Using tool: {block.name}")
            print(f"   Input: {block.input}")
        elif block.type == "text":
            print("\n📋 Final Plan:")
            display(Markdown(block.text))

# Note: This example shows the structure but won't execute the tool
# In a real application, you'd handle tool execution
thinking_with_tools_example()

<a id='best-practices'></a>
## 6. Best Practices

### 6.1 Choosing the Right Budget

In [ ]:
def budget_comparison():
    """Compare different thinking budgets"""
    
    problem = """Analyze this business scenario:
    A coffee shop has 3 locations. Location A makes $2,500/day, 
    Location B makes $1,800/day, and Location C makes $3,200/day.
    Operating costs are 65% of revenue. They want to open a 4th location.
    What factors should they consider and what's the minimum daily revenue 
    the new location needs to be profitable?"""
    
    budgets = [1024, 5000, 15000]
    
    print("💰 Thinking Budget Comparison")
    print("=" * 60)
    
    for budget in budgets:
        print(f"\n📊 Budget: {budget:,} tokens")
        print("-" * 40)
        
        start_time = time.time()
        
        response = client.messages.create(
            model="claude-sonnet-4-20250514",
            max_tokens=1000,
            thinking={"type": "enabled", "budget_tokens": budget},
            messages=[{"role": "user", "content": problem}]
        )
        
        elapsed_time = time.time() - start_time
        
        # Get response length
        response_text = ""
        for block in response.content:
            if block.type == "text":
                response_text = block.text
        
        print(f"⏱️  Time: {elapsed_time:.2f} seconds")
        print(f"📝 Response length: {len(response_text)} characters")
        print(f"💡 Response preview: {response_text[:200]}...")

budget_comparison()

### 6.2 Effective Prompting for Extended Thinking

In [ ]:
def prompting_best_practices():
    """Demonstrate effective prompting strategies"""
    
    # Good prompt - clear, specific, structured
    good_prompt = """Analyze the following investment options and recommend the best choice:

Option A: Stock Portfolio
- Expected annual return: 8%
- Risk level: High
- Minimum investment: $10,000
- Liquidity: High (can sell anytime)

Option B: Real Estate
- Expected annual return: 6%
- Risk level: Medium
- Minimum investment: $50,000
- Liquidity: Low (takes months to sell)

Option C: Bonds
- Expected annual return: 4%
- Risk level: Low
- Minimum investment: $5,000
- Liquidity: Medium

Investor Profile:
- Age: 35
- Investment horizon: 15 years
- Risk tolerance: Medium
- Available capital: $75,000
- Goal: Retirement savings

Please provide:
1. Analysis of each option
2. Recommended allocation
3. Justification for your recommendation"""
    
    response = client.messages.create(
        model="claude-sonnet-4-20250514",
        max_tokens=3000,
        thinking={"type": "enabled", "budget_tokens": 12000},
        messages=[{"role": "user", "content": good_prompt}]
    )
    
    print("✅ Best Practices Example: Structured Investment Analysis")
    print("=" * 60)
    
    for block in response.content:
        if block.type == "text":
            display(Markdown(block.text))

prompting_best_practices()

### Key Prompting Tips:

1. **Be Specific**: Clearly state what you want analyzed
2. **Provide Context**: Include all relevant information
3. **Structure Your Input**: Use clear formatting and sections
4. **Define Success Criteria**: Specify what a good answer looks like
5. **Avoid Over-Prompting**: Don't tell Claude to "think step by step" - it already does!

<a id='examples'></a>
## 7. Real-World Examples

### 7.1 Complex Document Analysis

In [ ]:
def document_analysis_example():
    """Analyze a complex document with extended thinking"""
    
    # Simulated legal document excerpt
    document = """PURCHASE AGREEMENT - EXECUTIVE SUMMARY
    
    This Agreement is entered into as of January 15, 2024, between TechCorp Inc. 
    ("Buyer") and DataSystems LLC ("Seller").
    
    TERMS:
    1. Purchase Price: $45,000,000 (Forty-five million dollars)
    2. Payment Structure:
       - Initial Payment: $20,000,000 upon closing
       - Deferred Payment: $15,000,000 payable over 3 years
       - Performance Earnout: Up to $10,000,000 based on revenue targets
    
    3. Conditions Precedent:
       - Regulatory approval from FTC
       - No material adverse change in Seller's business
       - Retention of key employees (minimum 80% for 12 months)
    
    4. Representations and Warranties:
       - Seller warrants all intellectual property is free of encumbrances
       - Financial statements are accurate per GAAP
       - No pending litigation exceeding $500,000
    
    5. Termination Clauses:
       - Either party may terminate if closing doesn't occur by March 31, 2024
       - Buyer may terminate if due diligence reveals material issues
       - Break-up fee: $2,000,000 if Buyer terminates without cause
    """
    
    response = client.messages.create(
        model="claude-opus-4-20250514",
        max_tokens=4000,
        thinking={"type": "enabled", "budget_tokens": 20000},
        messages=[{
            "role": "user",
            "content": f"""Analyze this purchase agreement and identify:
            
            1. Key risks for the buyer
            2. Key risks for the seller
            3. Potential deal breakers
            4. Areas that need clarification
            5. Recommendations for both parties
            
            Document:
            {document}"""
        }]
    )
    
    print("📄 Complex Document Analysis with Extended Thinking")
    print("=" * 60)
    
    for block in response.content:
        if block.type == "thinking":
            print("\n🧠 Analysis Process (Summary):")
            print(block.thinking[:800] + "...\n")
        elif block.type == "text":
            print("📊 Detailed Analysis:")
            display(Markdown(block.text))

document_analysis_example()

### 7.2 Code Architecture Planning

In [ ]:
def architecture_planning_example():
    """Use extended thinking for software architecture decisions"""
    
    requirements = """Design a microservices architecture for an e-commerce platform with:
    
    Functional Requirements:
    - User authentication and profiles
    - Product catalog with search
    - Shopping cart and checkout
    - Order management and tracking
    - Payment processing
    - Inventory management
    - Review and rating system
    
    Non-Functional Requirements:
    - Handle 100,000 concurrent users
    - 99.9% uptime
    - Response time < 200ms for catalog
    - Scalable to 10x current load
    - Multi-region deployment
    - GDPR compliant
    
    Tech Stack Preferences:
    - Cloud-native (AWS/GCP/Azure)
    - Container-based deployment
    - Modern languages (Python/Go/Node.js)
    """
    
    response = client.messages.create(
        model="claude-opus-4-20250514",
        max_tokens=5000,
        thinking={"type": "enabled", "budget_tokens": 25000},
        messages=[{
            "role": "user",
            "content": f"""Create a detailed microservices architecture plan.
            
            Include:
            1. Service breakdown and responsibilities
            2. Communication patterns (sync/async)
            3. Data storage strategy
            4. Security considerations
            5. Deployment architecture
            6. Scaling strategy
            7. Monitoring and observability
            
            Requirements:
            {requirements}"""
        }]
    )
    
    print("🏗️ Microservices Architecture Planning")
    print("=" * 60)
    
    for block in response.content:
        if block.type == "text":
            display(Markdown(block.text))

architecture_planning_example()

<a id='performance'></a>
## 8. Performance and Cost Considerations

### Understanding Token Usage and Costs

In [ ]:
def cost_calculator():
    """Calculate costs for extended thinking usage"""
    
    # Pricing as of the documentation (prices per million tokens)
    pricing = {
        "claude-opus-4": {"input": 15, "output": 75},
        "claude-sonnet-4": {"input": 3, "output": 15},
        "claude-sonnet-3.7": {"input": 3, "output": 15}
    }
    
    print("💰 Extended Thinking Cost Calculator")
    print("=" * 60)
    
    # Example scenario
    scenarios = [
        {"name": "Simple Analysis", "input": 500, "thinking": 5000, "output": 1000},
        {"name": "Complex Problem", "input": 2000, "thinking": 20000, "output": 3000},
        {"name": "Deep Research", "input": 5000, "thinking": 50000, "output": 8000}
    ]
    
    for model, prices in pricing.items():
        print(f"\n📊 Model: {model}")
        print("-" * 40)
        
        for scenario in scenarios:
            # Remember: thinking tokens are billed as output tokens
            input_cost = (scenario["input"] / 1_000_000) * prices["input"]
            thinking_cost = (scenario["thinking"] / 1_000_000) * prices["output"]
            output_cost = (scenario["output"] / 1_000_000) * prices["output"]
            total_cost = input_cost + thinking_cost + output_cost
            
            print(f"\n  {scenario['name']}:")
            print(f"    Input tokens: {scenario['input']:,}")
            print(f"    Thinking tokens: {scenario['thinking']:,} (billed as output)")
            print(f"    Output tokens: {scenario['output']:,}")
            print(f"    Total cost: ${total_cost:.4f}")

cost_calculator()

### Performance Optimization Tips

In [ ]:
def performance_tips():
    """Demonstrate performance optimization strategies"""
    
    print("⚡ Performance Optimization Strategies")
    print("=" * 60)
    
    strategies = [
        {
            "title": "1. Start with Minimal Budget",
            "description": "Begin with 1,024 tokens and increase only if needed",
            "example_budget": 1024,
            "use_case": "Simple calculations or basic analysis"
        },
        {
            "title": "2. Use Streaming for Better UX",
            "description": "Stream responses to show progress during long thinking",
            "example_budget": 10000,
            "use_case": "Interactive applications"
        },
        {
            "title": "3. Batch Processing for Large Budgets",
            "description": "Use batch API for thinking budgets > 32k tokens",
            "example_budget": 50000,
            "use_case": "Overnight analysis jobs"
        },
        {
            "title": "4. Cache Common Patterns",
            "description": "Use prompt caching for repeated analysis patterns",
            "example_budget": 15000,
            "use_case": "Standardized document analysis"
        }
    ]
    
    for strategy in strategies:
        print(f"\n{strategy['title']}")
        print(f"  📝 {strategy['description']}")
        print(f"  💡 Budget: {strategy['example_budget']:,} tokens")
        print(f"  🎯 Best for: {strategy['use_case']}")
    
    print("\n\n📈 Budget vs. Quality Guidelines:")
    print("-" * 40)
    print("  1,024 - 5,000 tokens: Basic reasoning tasks")
    print("  5,000 - 15,000 tokens: Standard complex problems")
    print("  15,000 - 32,000 tokens: Deep analysis and research")
    print("  32,000+ tokens: Extensive multi-faceted problems")

performance_tips()

## Summary and Next Steps

### What We've Learned

1. **Extended Thinking Basics**: How to enable and use Claude's reasoning capabilities
2. **Thinking Blocks**: Understanding the structure and content of thinking outputs
3. **Advanced Features**: Streaming, tool use, and complex scenarios
4. **Best Practices**: Optimal prompting and budget selection
5. **Real-World Applications**: Document analysis and architecture planning
6. **Cost Management**: Understanding pricing and optimization strategies

### When to Use Extended Thinking

✅ **Use it for:**
- Complex multi-step problems
- Deep document analysis
- Strategic planning and decision-making
- Quality-critical tasks where accuracy matters more than speed

❌ **Avoid it for:**
- Simple queries or lookups
- Real-time chat applications
- Tasks where latency is critical
- High-volume, low-complexity requests

### Resources for Further Learning

- [Anthropic Documentation](https://docs.anthropic.com/)
- [Extended Thinking Cookbook](https://docs.anthropic.com/cookbook/extended-thinking)
- [API Reference](https://docs.anthropic.com/api/)

### Try It Yourself!

Now that you understand extended thinking, try these challenges:

1. **Math Challenge**: Use extended thinking to solve a complex optimization problem
2. **Analysis Challenge**: Analyze a dataset and provide insights with reasoning
3. **Planning Challenge**: Design a system architecture for your own project
4. **Comparison Challenge**: Compare different thinking budgets on the same problem

Happy thinking! 🤔✨